In [1]:
import requests
import re
import time
import lxml.html
import openpyxl
import pandas as pd

from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
custom_header = {
    'referer' : 'http://www.riss.kr/index.do',
    #'referer' : 'http://www.riss.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5&exQuery=&exQueryText=&order=&onHanja=&strSort=&p_year1=&p_year2=&iStartCount=0&orderBy=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=bib_t&colName=bib_t&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&query=%EC%9D%B8%EA%B3%B5%EC%A7%80%EB%8A%A5',
    'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'  }

# 검색키워드: Satisfaction

In [3]:
#http://www.riss.or.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=Satisfaction&exQuery=regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88pyear%3A2021%E2%97%88pyear%3A2020%E2%97%88pyear%3A2019%E2%97%88pyear%3A2018%E2%97%88pyear%3A2017%E2%97%88pyear%3A2016%E2%97%88pyear%3A2015%E2%97%88pyear%3A2014%E2%97%88pyear%3A2013%E2%97%88pyear%3A2012%E2%97%88pyear%3A2011%E2%97%88pyear%3A2010%E2%97%88language%3Akor%E2%97%88kdc%3A5%E2%97%88&exQueryText=%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2021%5D%40%40pyear%3A2021%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2020%5D%40%40pyear%3A2020%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2019%5D%40%40pyear%3A2019%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2018%5D%40%40pyear%3A2018%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2017%5D%40%40pyear%3A2017%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2016%5D%40%40pyear%3A2016%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2015%5D%40%40pyear%3A2015%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2014%5D%40%40pyear%3A2014%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2013%5D%40%40pyear%3A2013%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2012%5D%40%40pyear%3A2012%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2011%5D%40%40pyear%3A2011%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2010%5D%40%40pyear%3A2010%E2%97%88%EC%9E%91%EC%84%B1%EC%96%B8%EC%96%B4+%5B%ED%95%9C%EA%B5%AD%EC%96%B4%5D%40%40language%3Akor%E2%97%88%EC%A3%BC%EC%A0%9C%EB%B6%84%EB%A5%98+%5B%EA%B8%B0%EC%88%A0%EA%B3%BC%ED%95%99%5D%40%40kdc%3A5%E2%97%88&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount=0&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=Satisfaction

In [4]:
paper_link_list = []
#160
for i in range(160):
    before_url = "http://www.riss.or.kr/search/Search.do?isDetailSearch=N&searchGubun=true&viewYn=OP&queryText=&strQuery=Satisfaction&exQuery=regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88pyear%3A2021%E2%97%88pyear%3A2020%E2%97%88pyear%3A2019%E2%97%88pyear%3A2018%E2%97%88pyear%3A2017%E2%97%88pyear%3A2016%E2%97%88pyear%3A2015%E2%97%88pyear%3A2014%E2%97%88pyear%3A2013%E2%97%88pyear%3A2012%E2%97%88pyear%3A2011%E2%97%88pyear%3A2010%E2%97%88language%3Akor%E2%97%88kdc%3A5%E2%97%88&exQueryText=%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EB%93%B1%EC%9E%AC%E2%97%88%EB%93%B1%EC%9E%AC%EC%A0%95%EB%B3%B4+%5BKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%5D%40%40regnm%3AKCI%EC%9A%B0%EC%88%98%EB%93%B1%EC%9E%AC%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2021%5D%40%40pyear%3A2021%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2020%5D%40%40pyear%3A2020%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2019%5D%40%40pyear%3A2019%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2018%5D%40%40pyear%3A2018%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2017%5D%40%40pyear%3A2017%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2016%5D%40%40pyear%3A2016%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2015%5D%40%40pyear%3A2015%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2014%5D%40%40pyear%3A2014%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2013%5D%40%40pyear%3A2013%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2012%5D%40%40pyear%3A2012%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2011%5D%40%40pyear%3A2011%E2%97%88%EB%B0%9C%ED%96%89%EC%97%B0%EB%8F%84+%5B2010%5D%40%40pyear%3A2010%E2%97%88%EC%9E%91%EC%84%B1%EC%96%B8%EC%96%B4+%5B%ED%95%9C%EA%B5%AD%EC%96%B4%5D%40%40language%3Akor%E2%97%88%EC%A3%BC%EC%A0%9C%EB%B6%84%EB%A5%98+%5B%EA%B8%B0%EC%88%A0%EA%B3%BC%ED%95%99%5D%40%40kdc%3A5%E2%97%88&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=&p_year2=&iStartCount="
    page = i*10
    after_url = "&orderBy=&mat_type=&mat_subtype=&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&image_yn=&gubun=&kdc=&ttsUseYn=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=re_a_kor&colName=re_a_kor&pageScale=10&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=Satisfaction"
    
    url = before_url + str(page) + after_url
    
    page_req = requests.get(url)
    page_html = page_req.text
    page_soup = BeautifulSoup(page_html, 'html.parser')

    #links.append(page_soup.select('#divContent > div.rightContent > div > div.srchResultW > div.srchResultListW > ul > li > div.cont > p.title > a'))
    links = page_soup.select('#divContent > div.rightContent > div > div.srchResultW > div.srchResultListW > ul > li > div.cont > p.title > a')
    
    for link in links:
        ## Tag안의 텍스트
        print(link.text)
        paper_link_list.append(link.get('href'))
    print(len(paper_link_list))

Effect of Self-leadership Recognized by Newly-employed Nurses on Job Satisfaction: Mediating Effect of Organizational Commitment
Patient Satisfaction, Vaginal Bleeding, Sexual Function following Laparoscopic Supracervical Hysterectomy
ORIGINAL ARTICLE : A Study on the Effects of Hospital Internal Marketing Factors on the Internal Customer Satisfaction
Original Article : The influence of general characteristics of physical therapy students in regards to major satisfaction and academic achivement
Transactions : Consumer Acceptance of Cosmetics Labels and Satisfaction Level with Cosmetics after Purchase: Viewed by the Type of Distribution Channel
Original Article : Satisfaction with the Quality of Agricultural Machinery and the Propensity for Replacement Purchases
Transactions : The Predictability of Emotional Labor Dimensions on Job Stress, Customer Orientation, and Job Satisfaction
Original Articles : The Efficacy of Discharge Instructions Provided by a Tablet Personal Computer for Aidi

임부의 결혼만족도와 배우자 지지가 태교실천에 미치는 영향
연구논문 : 전통민속마을 방문의 추진 요인 및 유인 요인이 만족도에 미치는 영향 연구 -경주 양동민속마을을 대상으로-
정신간호사의 정신간호사 이미지와 직무만족도 및 자기표현의 상관관계
지역축제의 서비스품질이 방문객 만족도 및 충성도에 미치는 영향 : 대구 약령시 한방축제를 대상으로
임산부의 결혼만족도, 부모됨의 태도와 태교실천에 관한 연구
노인의 우울, 영양상태 및 인지기능이 삶의 만족도에 미치는 영향
이단계 군집분석을 이용한 지출별 산악승마 만족 분석
연구논문 : 미용실의 불평행동요인이 불평처리 후 만족도와 사후행동에 미치는 영향
간호일터영성이 병원간호사의 직무만족, 소진 및 이직의도에 미치는 영향
연구논문 : 미용실의 관계혜택이 고객만족 및 고객충성도에 미치는 영향
160
연구논문 : 레스토랑의 서비스 제공단계별 Meal duration이 고객의 서비스품질 평가 및 고객만족에 미치는 영향
연구논문 : 미용 경락마사지에 대한 고객의 인지도 및 이용실태와 만족도
연구논문 : DiSC 행동유형이 미용전공대학생의 교육 만족도와진로결정 자기효능감에 미치는 영향
산촌주민의 산림노동 활동 증대를 위한 산림노동자의 직무스트레스와 직무만족 조사
스쿠버다이빙의 교육 서비스품질과 고객만족 및 재구매의사의 관계
어린이집 원장의 감성리더십과 보육교사의 이직의도와의 관계에서 직무만족도의 매개효과 검증
일 종합병원 외래환자의 진료대기시간 및 환자만족도, 재이용의도와의 관계
서울시 아이돌보미의 직무 만족도에 관한 연구
한국과학영재학교 학생의 학교생활만족도: 생태학적 접근
연구논문 : 동기위생이론과 Kano 모델을 적용한 소셜커머스의 만족과 불만족 차원 연구
170
연구논문 : 헤어미용종사자의 신체적 매력, 자아존중감 및 대인관계가 직무만족도에 미치는 영향
연구논문 : 뷰티산업의 감성 서비스 요인별 고객만족도
내부마케팅이 치과위생사의 직무만족과 이직의도에 미치는 영향
커피전문점의 실내디자인 요소 변화에 따른 공간 만족도 분석
원저

간호전달체계의 변화가 간호사 만족도와 환자만족도 및 직접간호시간에 미치는 효과
한식 식단 급식을 제공받는 경기도의 일부 중학생과 학부모의 한식에 대한 인식 및 급식 만족도
간호사의 팔로워십과 조직몰입, 직무만족, 이직의도 및 고객지향성간의 관계
보육교사의 이직의도에 대한 조직문화와 직무만족도의 영향
청소년 삶의 만족도에 대한 공부 중요성 인식, 공부압력과 성공압력의 영향력 및 부모-자녀 의사소통의 중재효과
대안관광종사원의 직무교육이 직무만족에 미치는 영향
논문 : 슬로우 투어리즘 관광객의 이용행태,관광도기,만족도와 행동의도간의 관계 -대구올레 팔공산 코스를 중심으로-
요양병원과 요양시설 노인의 건강상태, 사회적 지지 및 거주만족도
포괄간호서비스를 제공하는 간호사의 간호업무와 직무 스트레스, 직무만족에 관한 연구
연구논문 : 천연염색제품 구매 평가기준 및 구매 후 만족도
350
치과의료서비스의 질과 환자만족도와의 연관성
치과위생사의 의료서비스에 따른 내원환자의 만족도
보문 : 인터넷 쇼핑몰 유형별 패션 소비자의 서비스 회복 공정성 지각, 구매 만족도, 긍정적 구전의도 및 재구매의도에 관한 연구
성기능증진 프로그램이 유방암 생존 여성의 성스트레스, 성만족 및 부부친밀감에 미치는 효과
웰빙 라이프스타일, 웰빙태도, 삶의 만족도 및 인구통계학적 특성간의 관계
대구 외식업 종사원의 직무소진에 관한 연구 -"가족-일 갈등", "일-가족 갈등", 직무만족 및 개인직무성과를 중심으로-
기혼직장인의 자아분화가 가정생활만족과 직무만족에 미치는 영향 : 갈등대처행동을 매개변인으로
연구논문 : 미용실의 관리적 측면이 고객만족 및 재방문의도에 미치는 영향 연구
병원 간호사의 임파워먼트가 간호업무성과, 직무만족, 조직몰입, 이직의도에 미치는 영향
대학생의 성격강점과 의사소통유형이 대인관계만족도에 미치는 영향
360
기업 소비자상담사의 블랙컨슈머 대응행동과 업무스트레스가 업무수행과 업무만족도에 미치는 영향
연구논문 : 면세점 이용 소비자의 점포 추구혜택 및 패션제품 쇼핑성향이 점포 만족

원저 : 산업체 위탁생의 계속교육에 관한 만족도
패션관광특구 방문객의 패션 쇼핑 특성 및 만족도 비교
한약재 구매자-공급자 협력에 관한 탐색적 연구 -만족, 보상력, 신뢰를 중심으로-
입체 패널의 형태 유형에 따른 이용자 만족도 평가
인터넷 일반쇼핑몰과 여성의류쇼핑몰의 선호요인과 만족도
국립공원 탐방행태 및 이용만족요인 -경주국립공원을 사례로-
본문 : e-CRM 활동이 고객만족 및 재구매의도에 미치는 영향 -남녀 대학생을 중심으로-
장애인체육 종사자들의 직무 만족 및 불만족 요인
오픈마켓 의류구매에서의 제품 및 쇼핑몰 속성 만족이 구매행동에 미치는 영향 : 쇼핑몰 신뢰, 재구매 의도, 추천 의도를 중심으로
보문 : 병,의원의 의료서비스와 환자복에 대한 소비자 만족
540
광주·전남 지역 학교급식 영양사·영양교사의 학교유형에 따른 직무분석 및 만족도 연구
보문 : 국내/해외 SPA브랜드 의복구매 20대 여성 소비자의 쇼핑성향과 의복만족도
연구논문 : 남자경찰보호복 착용실태 및 만족도에 관한 연구
경기도 소재 특수학교에 근무하는 영양사,영양교사의 직무만족도와 급식운영 현황
인터넷 쇼핑몰입이 고객만족과 충성도에 미치는 영향에 관한 연구
군집분석을 이용한 응급실 간호사-환자 상호작용 유형 및 환자의 상호작용 만족도
연구논문 : 한국에 거주하는 이주여성의 외모관심도와 외모만족도가 뷰티관리행동에 미치는 영향
국민주택규모 아파트 단위주거 공간에 대한 자녀들의 공간만족도에 대한 연구 : 건축학과 대학생을 중심으로
연구논문 : 부산지역 대학교 급식소의 급식서비스 만족도와 고객충성도와의 관계
재가노인의 건강상태 인지 수준에 따른 생활만족도의 영향요인
550
병원간호사의 소진, 직무만족, 조직몰입과 이직의도의 관계
BMI지수에 의한 신체유형별 신체만족도와 의복적합성에 관한 연구: 20~50대 여성을 중심으로
일부 대학부속병원 외래간호사와 병동간호사의 업무 스트레스와 직무만족도 비교
외암민속마을의 주생활 특성 및 주거 만족도에 관한 연구 : 민속마을 보존정책을 중심으로
와인교육프

스리랑카 사무르디 빈곤퇴치프로그램 수혜자의 사회경제적 조건과 만족도 분석
과제 지향적 활동이 만성 뇌졸중 환자의 일상생활동작 수행능력과 균형능력 및 만족도에 미치는 영향
연구논문 : 농촌체험마을 방문객 선택속성에 대한 만족과 재방문의사에 미치는 영향
만족 극대화 측면의 경제적 접근에 따른 분임조 활동 분석
연구논문 : 근린공원 시설유형에 따른 이용실태 및 만족도에 관한 연구
6시그마를 활용한 프랜차이즈업의 고객만족도 향상에 관한 연구
연구논문 : 미용관련학과 학생들의 취업의식과 학과만족도에 관한 연구
원저 : 노인의 주관적 구강건강인식과 의치만족도
협진 의뢰 산모의 한방산후조리 현황 분석 및 만족도 조사
우리나라 화훼장식 교육프로그램 분석과 화훼장식가와 지망생 직업만족도 비교
740
지역별 치위생과 학생들의 교육환경 및 전공 만족도 조사
원저 : 일부 치위생과 학생의 학과선택 동기 및 적응도
스마트폰 구매결정 요인이 고객만족에 미치는 영향 : 연령과 성별 조절 효과 분석
연구논문 : 도심지 내 명승 주변지역 거주민의 의식 연구 -명승 제35호 성락원(城樂園) 주변 역사문화환경 보전지역을 중심으로-
지역사회 청소년에서 체중과 자아존중감, 우울 및 불안 간의 관계
옥상정원 구성요소와 이용 후 평가
임상실습이 치위생 전공학생의 전문직 정체성에 미치는 영향
요양원 뇌졸중 노인의 생활만족도 향상을 위한 원예치료 프로그램의 활용
스포츠 유틸리티 차량의 테일게이트 개폐 불편도와 만족도 평가
중학생의 여가활동과 흡연·음주와의 관계
750
Original Articles : Critical Pathway Implementation for Minor Motor Vehicle Trauma Patients
Original Article : 어머니의 양육 스트레스, 갈등대처행동 및 결혼만족도가 유아의 위축행동에 미치는 영향
대형선망어업의 내부마케팅이 선원태도에 미치는 영향에 관한 연구
Kano 모델 기반의 잠재적 고객 요구 개선 지수의 개발 : 교육서비스 분야를 중심으로
연구논문 :

단계별 주민역량강화를 통한 농촌마을만들기의 효과 연구 -전라북도 진안군을 대상으로-
외과계 간호사를 위한 환자안전 관점의 실무교육 프로그램 개발과 효과
원저 :  응급의료전용 헬기 탑승자 교육프로그램 개발 및 효과
암을 처음 진단받은 환자를 위한 신환 네비게이션 프로그램 개발 및 효과 평가
흉터재생침과 미세피부침을 이용한 여드름 흉터의 치험례
일개 종합병원 전자간호기록시스템 도입 후 중기 평가연구
Long-term Results of Stellate Ganglion Block in Patients with Olfactory Dysfunction
보육시설 조직커뮤니케이션과 갈등, 몰입에 관한 연구
원저 : 응급실 기반 소아 가정 내 손상의 시범적 예방 교육 적용의 효과
SBAR를 이용한 의사소통이 간호사의 의사소통 인식과 환자안전에 대한 태도에 미치는 효과
940
연구논문 : 미국산 쇠고기 선택속성의 중요도-만족도 분석 -서울,수도권 대학생을 중심으로-
ORiginal Article : Sodium Picosulfate with Magnesium Citrate (SPMC) Plus Laxative Is a Good Alternative to Conventional Large Volume Polyethylene Glycol in Bowel Preparation: A Multicenter Randomized Single-Blinded Trial
간호대학생의 성격유형, 학업 스트레스, 스트레스 대처방식이 대학생활 적응에 미치는 영향
노인 불면에 대한 EFT 불면 치료 프로그램(EFT-1)의 효과 평가를 위한 예비적 연구
3년제 간호대학 신입생의 자아존중감, 정서지능, 사회적 지지가 대학생활 적응에 미치는 영향
원저 : 피부분석기기(Clarity(TM) Skin Advisor System)의 임상적 유용성에 관한 연구
기술사용이 근로환경과 근로자 건강에 미치는 영향력에 대한 연구
중소 병원 간호사들의 업무상 손상경험에 영향을 미치는 요인파악
원저 : 신장이식 1개월 이

한국 도시공원의 베리어 프리 정비사업의 정책제언을 위한 일본 도시공원 정비사례에 대한 평가
원저 : 레이저 시술 후 발생한 피부 장벽 손상과 자극 증상에 대한 APDDR-1001 제제 국소 도포의 효과
연구논문 : 공군 정비파카의 기능성 개선을 위한 실태조사
건강증진 운동정보판(EBHP) 개발을 위한 실효성 검증 : 대구경북지역을 중심으로
Transactions : The Effect of Counterfeits on the Perceptions toward Luxury Fashion Brands
원예재활실습 만족도가 화훼원예전공생의 관련분야 직업준비의사에 미치는 영향
컬러데이터베이스를 이용한 선호도기반컬러 선택인터페이스
자원봉사활동을 하는 일개 간호대학 학생들의 노인에 대한 태도 영향요인
요실금 여성의 성기능에 영향을 미치는 요인
야간전담간호사와 3교대간호사의 표준 교대근무 지표(Standard Shiftwork Index) 차이
1120
원저 : 2% 알부틴 함유 하이드로겔 마스크의 기미개선 효과에 관한 연구
Perceived Efficacy and Goal Setting System과 자기효능감 검사의 상관관계 연구
원저 : 뇌사 장기 기증자 가족의 장기 기증에 대한 긍정성 조사
보행권 확보를 위한 보행안전대책의 도입방안에 관한 기초적 연구 : 선진국 보행안전대책을 중심으로
원예활동이 갱년기 도시주부의 스트레스 경감에 미치는 효과
연구논문(硏究論文) : 공원이용자 의식조사를 통한 에너지 절약형 조경시설물 도입에 대한 연구 -김포시 근린공원을 중심으로-
여가활동으로서의 무도스포츠 역할 재고를 위한 고찰
공항 서비스 향상을 위한 지능형 연령인식 디지털 사이니지 연구
한국판 수용인정척도의 타당화 : 데이트 폭력 피해 여대생을 대상으로
앱게임 인터페이스 핵심 디자인요소에 대한 만족도 평가 방법 연구
1130
IPA 분석을 이용한 간선급행버스 이용자 서비스 특성분석 : 용인 수지지구 중심으로
QoE를 이용한 네트워크 콘텐츠 서비스 수익모델 설계 및 분석
주관절 외

원저 : 개원의로 이직한 응급의학과 전문의
신규간호사의 직무 스트레스, 자아존중감, 회복탄력성이 현장적응에 미치는 영향
연구논문(硏究論文) : 필름 느와르에 나타난 팜므 파탈의 복식유형 연구 -이중배상<(1944)>을 중심으로-
노인요양시설의 환자안전문화 인식에 관한 귀납적 내용분석
원저 : 삼기음가감방과 봉약침, 체침의 병행치료가 뇌졸중 후 견관절 통증에 미치는 효과 비교연구
항공사 객실승무원의 개인적 감성능력과 사회적 감성능력이 팀웍역량에 미치는 영향에 관한 연구
연구논문 : 횡성축협한우의 구매행동에 영향을 미치는 요인 분석
철강 제조업 근로자의 직무스트레스 분석 및 대책 방안
Polymannuronate 첨가에 의한 햄버거빵이 흰쥐의 지질대사에 미치는 영향
원가족 분화경험이 기혼남녀의 결혼적응에 미치는 영향: 불안과 행복감을 매개로
1310
폐경기 여성의 혈관운동성 증상과 체열분포, 신허증 변증유형의 상관성 연구
전문간호사의 성과지표 개발
여성암환자의 운동프로그램 운영 및 참여경험에 관한 사례연구
간호대학생의 건강증진 생활양식에 영향을 미치는 요인
생명과학 캠프에 참여한 초·중등학생의 학년별 프로그램 만족도
제천시 한방 난임 치료 지원 사업에 관한 연구
User Acceptance of a Light-Emitting Diode Vest for Police Officer
간호사의 환자참여 간호서비스 수행정도와 필요성 인식
간호사의 이직의도 예측모형
경영회생지원 농지매입사업 이용실태에 관한 사례연구: 강원지역 농가를 중심으로
1320
남성 알코올 중독자의 단주 경험
다문화가족 청소년의 개인적, 심리적, 건강행위와 주관적 건강상태
연구논문 : 인터넷 의류쇼핑몰 상품 사진의 표현형식과 배경이 소비자의 지각에 미치는 영향
스마트폰 구매 시 이동통신사 선정에 영향을 미치는 요인에 대한 AHP 분석
스마트 폰을 이용한 SFA 시스템 구축에 관한 연구
한국 중년 여성의 실제이미지와 이상적이미지에 대한 연구 : 연령별 비교를 중심으로
연구논문 : 스마트폰 관련 규제정책이

외래환자들의 복지용구 사용현황 및 교육 필요성
조직 커뮤니케이션 매체로서 위성영상방송 : 정보적 · 관계적 차원의 효과를 중심으로
장애인 환자를 위한 구강의료 서비스지침 개발의 전략적 방향 - IPA 기법을 중심으로 -
리서치기반의 디자인 교육의 효과성 검증
소득계층 인식과 정치적 성향이 개인의 환경보호에 대한 태도에 미치는 영향
증례를 통해 본 치매의 한양방 협진 모델 연구
연구논문 : 환경친화집단과 비환경친화집단의 친환경 의류제품에 대한 소비자 특성연구-대전지역 대학생을 중심으로-
연구논문 : 성인 여성의 메이크업 교육 경험에 따른 교육 필요성 및 교육 프로그램 요구도에 관한 연구
베타-3 아드레날린 수용체 유전자 다형성에 따른 대학생의 생화학 지표, 체성분과 편의식품 섭취실태에 관한 연구
도시공원의 운영 및 관리에 대한 지역주민과 관광객 간 인식 차이 연구 -인천광역시 월미공원을 중심으로-
1510
산업장 근로자의 스트레스에 대한 한의계 연구경향
클라이언트중심 가정방문 작업치료가 치매특별등급 대상자의 활동참여에 미치는 효과 : 사례연구
보문 : 웨어러블 테크놀로지에 기반을 둔 고기능 스마트 재킷 설계 제안
The Effects of the Talent Types on Purchasing Behavior of Beauty-Service products
비선형동적해석을 통한 국내 철골 모멘트골조의 내진성능 평가
Effect of Heated Red Bean Pillow Application for College Women with Dysmenorrhea
족부 족관절에서 발생한 상피낭종
예술의 전당 오페라 극장의 주관적 음향성능 평가
한국 간호사의 회복탄력성과 관련된 변인의 메타분석
다층모형을 활용한 상수원 관리지역 주민지원사업 평가에 관한 연구
1520
원저 : 한국인 기미 환자의 삶의 질 평가에 대한 Melasma Quality of Life Scale (MELASQOL)과 Dermatology Life Quality Index (DLQI)의 비교 연구
대

In [5]:
paper_link_list

['/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=37bcefa1d84b09b94884a65323211ff0',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=b7cb0cdcc45894916aae8a972f9116fb',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=483d811420d951fbb7998d826d417196',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=87d5d857c3c82a70ffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=a82188135a8ce796ffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=e6283dff3ac52cfcb7998d826d417196',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=ad8fdfbe66aff8b6ffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=645a247f828d0ab0ffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=809248725ab85040ffe0bdc3ef48d419',
 '/search/detail/DetailView.do?p_mat_type=1a0202e37d52c

In [6]:
len(paper_link_list)

1598

In [7]:
paper_url = []

for paper_link in tqdm(paper_link_list):
    tmp_url = "http://www.riss.kr" + paper_link
    paper_url.append(tmp_url)

100%|█████████████████████████████████████████████████████████████████████████| 1598/1598 [00:00<00:00, 1602318.38it/s]


In [8]:
paper_url

['http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=37bcefa1d84b09b94884a65323211ff0',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=b7cb0cdcc45894916aae8a972f9116fb',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=483d811420d951fbb7998d826d417196',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=87d5d857c3c82a70ffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=a82188135a8ce796ffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=e6283dff3ac52cfcb7998d826d417196',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=ad8fdfbe66aff8b6ffe0bdc3ef48d419',
 'http://www.riss.kr/search/detail/DetailView.do?p_mat_type=1a0202e37d52c72d&control_no=645a247f828d0ab0ffe0bdc3ef48d419',
 'http://www.ris

In [9]:
len(paper_url)

1598

In [10]:
write_wb = openpyxl.Workbook()
write_ws = write_wb.active

In [11]:
"""
write_ws = write_wb.active
write_ws['A1'] = 'title_kor'
write_ws['B1'] = 'title_eng'
write_ws['C1'] = 'author'
write_ws['D1'] = 'book'
write_ws['E1'] = 'year'
write_ws['F1'] = 'keyword'
write_ws['G1'] = 'abstract_kor'
write_ws['H1'] = 'abstract_eng'
"""

"\nwrite_ws = write_wb.active\nwrite_ws['A1'] = 'title_kor'\nwrite_ws['B1'] = 'title_eng'\nwrite_ws['C1'] = 'author'\nwrite_ws['D1'] = 'book'\nwrite_ws['E1'] = 'year'\nwrite_ws['F1'] = 'keyword'\nwrite_ws['G1'] = 'abstract_kor'\nwrite_ws['H1'] = 'abstract_eng'\n"

In [12]:
write_ws = write_wb.active
write_ws['A1'] = 'title'
write_ws['B1'] = 'author'
write_ws['C1'] = 'book'
write_ws['D1'] = 'year'
write_ws['E1'] = 'keyword'
write_ws['F1'] = 'abstract_kor'
write_ws['G1'] = 'abstract_eng'

In [13]:
start = time.time()

for i, url in enumerate(paper_url):
    res = requests.get(url, headers = custom_header)
    element = lxml.html.fromstring(res.text)

    try:
        tmp_title = element.cssselect('h3.title')
        tmp_title = tmp_title[0].text_content()
        tmp_title = tmp_title.split("=")
        title_kor = tmp_title[0].strip()
        #title_eng = tmp_title[1].strip()

        tmp_author = element.cssselect('div.infoDetailL > ul > li:nth-child(1) > div > p')
        author = tmp_author[0].text_content().strip()

        tmp_book = element.cssselect('div.infoDetailL > ul > li:nth-child(3) > div > p')
        book = tmp_book[0].text_content().strip()

        tmp_year = element.cssselect('div.infoDetailL > ul > li:nth-child(5) > div > p')
        year = tmp_year[0].text_content().strip()

        tmp_keyword = element.cssselect('div.infoDetailL > ul > li:nth-child(7) > div > p')
        keyword = tmp_keyword[0].text_content().strip()

        tmp_abstract = element.cssselect('div.text.off')
        abstract_kor = tmp_abstract[0].text_content().strip()
        abstract_eng = tmp_abstract[1].text_content().strip()

        print(i, 'paper clear')

    except:
        print(i, 'ERROR')
        continue


    write_ws.append([title_kor, author, book, year, keyword, abstract_kor, abstract_eng])
    write_wb.save('./8_1_Satisfaction.xlsx')
    
print("Running Time : ", time.time() - start)

0 ERROR
1 ERROR
2 ERROR
3 ERROR
4 ERROR
5 ERROR
6 ERROR
7 ERROR
8 ERROR
9 ERROR
10 ERROR
11 ERROR
12 ERROR
13 ERROR
14 ERROR
15 ERROR
16 ERROR
17 ERROR
18 paper clear
19 ERROR
20 paper clear
21 ERROR
22 ERROR
23 ERROR
24 ERROR
25 ERROR
26 ERROR
27 paper clear
28 ERROR
29 paper clear
30 ERROR
31 ERROR
32 ERROR
33 paper clear
34 ERROR
35 paper clear
36 paper clear
37 ERROR
38 paper clear
39 paper clear
40 paper clear
41 paper clear
42 paper clear
43 paper clear
44 ERROR
45 ERROR
46 paper clear
47 paper clear
48 ERROR
49 ERROR
50 paper clear
51 paper clear
52 paper clear
53 paper clear
54 paper clear
55 ERROR
56 paper clear
57 ERROR
58 paper clear
59 paper clear
60 paper clear
61 ERROR
62 paper clear
63 ERROR
64 paper clear
65 paper clear
66 paper clear
67 ERROR
68 paper clear
69 paper clear
70 paper clear
71 paper clear
72 paper clear
73 paper clear
74 ERROR
75 ERROR
76 paper clear
77 ERROR
78 paper clear
79 paper clear
80 paper clear
81 paper clear
82 ERROR
83 paper clear
84 paper clear

605 paper clear
606 ERROR
607 paper clear
608 paper clear
609 ERROR
610 paper clear
611 paper clear
612 ERROR
613 ERROR
614 ERROR
615 paper clear
616 paper clear
617 paper clear
618 paper clear
619 paper clear
620 ERROR
621 ERROR
622 paper clear
623 ERROR
624 ERROR
625 paper clear
626 paper clear
627 ERROR
628 paper clear
629 paper clear
630 paper clear
631 ERROR
632 ERROR
633 ERROR
634 paper clear
635 paper clear
636 paper clear
637 ERROR
638 paper clear
639 ERROR
640 ERROR
641 paper clear
642 paper clear
643 ERROR
644 paper clear
645 paper clear
646 paper clear
647 ERROR
648 paper clear
649 paper clear
650 paper clear
651 ERROR
652 paper clear
653 paper clear
654 paper clear
655 ERROR
656 paper clear
657 paper clear
658 ERROR
659 paper clear
660 paper clear
661 ERROR
662 ERROR
663 ERROR
664 ERROR
665 paper clear
666 ERROR
667 paper clear
668 paper clear
669 ERROR
670 paper clear
671 ERROR
672 paper clear
673 ERROR
674 paper clear
675 paper clear
676 ERROR
677 ERROR
678 paper clear
67

1179 ERROR
1180 ERROR
1181 ERROR
1182 ERROR
1183 ERROR
1184 paper clear
1185 ERROR
1186 ERROR
1187 ERROR
1188 ERROR
1189 paper clear
1190 ERROR
1191 paper clear
1192 ERROR
1193 ERROR
1194 ERROR
1195 paper clear
1196 ERROR
1197 ERROR
1198 ERROR
1199 ERROR
1200 ERROR
1201 paper clear
1202 paper clear
1203 ERROR
1204 ERROR
1205 ERROR
1206 ERROR
1207 paper clear
1208 paper clear
1209 ERROR
1210 ERROR
1211 ERROR
1212 paper clear
1213 paper clear
1214 paper clear
1215 ERROR
1216 paper clear
1217 paper clear
1218 paper clear
1219 paper clear
1220 paper clear
1221 ERROR
1222 paper clear
1223 paper clear
1224 ERROR
1225 paper clear
1226 paper clear
1227 ERROR
1228 ERROR
1229 paper clear
1230 ERROR
1231 paper clear
1232 paper clear
1233 paper clear
1234 paper clear
1235 paper clear
1236 ERROR
1237 ERROR
1238 ERROR
1239 paper clear
1240 ERROR
1241 paper clear
1242 ERROR
1243 paper clear
1244 ERROR
1245 ERROR
1246 paper clear
1247 ERROR
1248 ERROR
1249 ERROR
1250 ERROR
1251 paper clear
1252 paper 